## Recommender Systems

In this notebook, I developed a sophisticated recommender system using a dataset of approximately 1950 articles scraped from various news sources. The objective was to create a personalized news recommendation engine utilizing three different approaches:

  **Content-Based Filtering**: This method recommends articles similar to a given article by analyzing the content and computing similarity scores using word embeddings. It focuses on the attributes of the articles themselves to find similarities.

  **Collaborative Filtering**: This approach generates recommendations based on user interactions and preferences. To simulate real-world user behavior, we created a user profile by selecting random topics for a user and designating these topics as their preferences. Articles were served based on these preferences, with clicks determined by a binomial distribution (click probability = 0.7). The time spent on each article was randomly decided, and ratings were assigned based on the time spent. Additionally, some articles outside the user's preferred topics were served with a lower click probability (0.1). This simulated scenario provided realistic data for evaluating the collaborative filtering approach.

  **Hybrid Recommender System**: To leverage the strengths of both content-based and collaborative filtering, I implemented a hybrid recommendation system. This system combines the content-based similarity scores with the collaborative filtering rankings to provide more accurate and personalized recommendations.

The hybrid approach integrates the advantages of both methods, offering a balanced and enhanced recommendation system. This project demonstrates the implementation and comparison of different recommendation techniques, showcasing their individual and combined capabilities in delivering personalized content to users.

# Importing data and lemmatizing the data

In [1]:
import numpy as np
import pandas as pd
import re
import scipy
import math

In [2]:
URL = 'https://drive.google.com/file/d/137eW4F35OctoRuq5DasVUXw6GpmfXdBS/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
#df = pd.read_pickle(path)
data = pd.read_csv(path, skip_blank_lines=True)
pd.set_option('display.max_colwidth', None)
print(data.shape)

(1950, 6)


In [3]:
data.drop_duplicates(subset='content', inplace=True, ignore_index=True)
data.shape

(1761, 6)

In [4]:
data.head(1)

Unnamed: 0                       _id  \
0           0  6076fad70b3e8bc9b779293a   

                                                                                            title  \
0  Australia's first international cricket team found fame in the UK. At home, they were betrayed   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
data[data['_id'] == '6076fadb0b3e8bc9b779293e']['_id'].to_string()

'4    6076fadb0b3e8bc9b779293e'

In [6]:
def make_lower_case(text):
    return text.lower()

In [7]:
import re
from pprint import pprint
import nltk, spacy, gensim
from sklearn.feature_extraction.text import CountVectorizer

def get_lemmatized_clean_data(df):
    # Convert to list
    data = df.content.tolist()

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    # pprint(data[:1])

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # print(data_words[:1])

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        return texts_out

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return data_lemmatized

In [8]:
X = get_lemmatized_clean_data(data)

In [9]:
max_time = []
for i in X:
  max_time.append(len(i.split(' '))/2.5)

data['Max_Time'] = max_time

# SKlearn NewsData Import

In [10]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [11]:
def get_data(mydata):
    mydata.keys()
    df = pd.DataFrame([mydata['data'],[mydata['target_names'][idx] for idx in mydata['target']],mydata['target']])
    df = df.transpose()
    df.columns = ['content', 'target_names', 'target']
    return df

df = get_data(newsgroups_train)

In [12]:
df.head()

,content,target_names,target
0,"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",rec.autos,7
1,"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n",comp.sys.mac.hardware,4
2,"From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985. sooo, i'm in the market for a\nnew machine a bit sooner than i intended to be...\n\ni'm looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected? i'd heard the 185c was supposed to make an\nappearence ""this summer"" but haven't heard anymore on it - and since i\ndon't have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo's just went through recently?\n\n* what's the impression of the display on the 180? i could probably swing\na 180 if i got the 80Mb disk rather than the 120, but i don't really have\na feel for how much ""better"" the display is (yea, it looks great in the\nstore, but is that all ""wow"" or is it really that good?). could i solicit\nsome opinions of people who use the 160 and 180 day-to-day on if its worth\ntaking the disk size and money hit to get the active display? (i realize\nthis is a real subjective question, but i've only played around with the\nmachines in a computer store breifly and figured the opinions of somebody\nwho actually uses the machine daily might prove helpful).\n\n* how well does hellcats perform? ;)\n\nthanks a bunch in advance for any info - if you could email, i'll post a\nsummary (news reading time is at a premium with finals just around the\ncorner... :( )\n--\nTom Willis \ twillis@ecn.purdue.edu \ Purdue Electrical Engineering\n---------------------------------------------------------------------------\n""Convictions are more dangerous enemies of truth than lies."" - F. W.\nNietzsche\n",comp.sys.mac.hardware,4
3,"From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nOrganization: Harris Computer Systems Division\nLines: 14\nDistribution: world\nNNTP-Posting-Host: amber.ssd.csd.harris.com\nX-Newsreader: TIN [version 1.1 PL9]\n\nRobert J.C. Kyanko (rob@rjck.UUCP)

In [13]:
news = data.drop(axis = 1, columns=['_id',]).to_numpy()

In [14]:
data_lemmatized = get_lemmatized_clean_data(df)

In [15]:
df.head()

,content,target_names,target
0,"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",rec.autos,7
1,"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n",comp.sys.mac.hardware,4
2,"From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985. sooo, i'm in the market for a\nnew machine a bit sooner than i intended to be...\n\ni'm looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected? i'd heard the 185c was supposed to make an\nappearence ""this summer"" but haven't heard anymore on it - and since i\ndon't have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo's just went through recently?\n\n* what's the impression of the display on the 180? i could probably swing\na 180 if i got the 80Mb disk rather than the 120, but i don't really have\na feel for how much ""better"" the display is (yea, it looks great in the\nstore, but is that all ""wow"" or is it really that good?). could i solicit\nsome opinions of people who use the 160 and 180 day-to-day on if its worth\ntaking the disk size and money hit to get the active display? (i realize\nthis is a real subjective question, but i've only played around with the\nmachines in a computer store breifly and figured the opinions of somebody\nwho actually uses the machine daily might prove helpful).\n\n* how well does hellcats perform? ;)\n\nthanks a bunch in advance for any info - if you could email, i'll post a\nsummary (news reading time is at a premium with finals just around the\ncorner... :( )\n--\nTom Willis \ twillis@ecn.purdue.edu \ Purdue Electrical Engineering\n---------------------------------------------------------------------------\n""Convictions are more dangerous enemies of truth than lies."" - F. W.\nNietzsche\n",comp.sys.mac.hardware,4
3,"From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nOrganization: Harris Computer Systems Division\nLines: 14\nDistribution: world\nNNTP-Posting-Host: amber.ssd.csd.harris.com\nX-Newsreader: TIN [version 1.1 PL9]\n\nRobert J.C. Kyanko (rob@rjck.UUCP)

# Converting the data to bag_of_word representation

In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [18]:
vectorizor = TfidfVectorizer(stop_words=my_stopwords, lowercase= True)
bag_of_words = vectorizor.fit_transform(X)

# Content Based Filtering

In this section, we delve into Content-Based Filtering techniques, a cornerstone of recommendation systems that analyze the intrinsic characteristics of items to make personalized recommendations. Unlike Collaborative Filtering, which relies on user interactions, Content-Based Filtering focuses on the features of items themselves to suggest similar items to users.

Exploring Various Techniques:

    TF-IDF Matrix: We begin by constructing a TF-IDF (Term Frequency-Inverse Document Frequency) matrix from our dataset of news articles. This matrix represents the importance of each term in each document, allowing us to measure the relevance of articles based on their textual content.

    SVD (Singular Value Decomposition): Utilizing SVD, a matrix factorization technique, we decompose the TF-IDF matrix into lower-dimensional matrices. This helps us identify latent factors that capture semantic relationships between articles, enhancing the accuracy of our recommendations.

    LDA (Latent Dirichlet Allocation): Implementing LDA, a probabilistic model, we uncover latent topics within the articles. By modeling the distribution of topics across articles and words within topics, LDA enables us to recommend articles based on thematic similarities, thereby improving content-based recommendations.

    GloVe Vectorizer: Leveraging GloVe (Global Vectors for Word Representation), a pretrained word embedding model, we transform text into dense vector representations. GloVe embeddings capture semantic relationships between words based on co-occurrence statistics, enabling us to measure similarities between articles more effectively.

Enhancing Recommendation Quality:
Each of these techniques offers unique advantages in capturing semantic relationships and thematic similarities within our news articles dataset. By combining these methods or selecting the most suitable approach based on our evaluation, we aim to enhance the relevance and personalization of our content-based recommendations.

Next, let's explore the implementation details and evaluation of these Content-Based Filtering techniques to understand their impact on our news recommender system.

## Content Based Similarity Using TFIDF Vector

In [19]:
from numpy import dot
from numpy.linalg import norm

def similarity(a,b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

In [20]:
def ContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(bag_of_words[news_index].todense())[0]
    b = np.array(bag_of_words[an_index].todense())[0]
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key =
             lambda kv:(kv[1], kv[0]), reverse=True)

  return sorted_most_similar[:first_n]

ContentBasedFiltering('6076fadb0b3e8bc9b779293e')


[('6076fadb0b3e8bc9b779293e', 1.0000000000000002),
 ('6076fb580b3e8bc9b7792991', 0.475870441754182),
 ('6076fbe10b3e8bc9b77929ec', 0.34154910566753116),
 ('6076fbef0b3e8bc9b77929f5', 0.20774470652516044),
 ('6077053f0b3e8bc9b7793016', 0.20312082641868645),
 ('607706000b3e8bc9b7793092', 0.1951150541596632),
 ('6077056b0b3e8bc9b7793032', 0.16203945976451536),
 ('6076fbd20b3e8bc9b77929e2', 0.1487360307853129),
 ('6076ff750b3e8bc9b7792c41', 0.14574688984806897),
 ('607705fb0b3e8bc9b7793091', 0.14275138882437455)]

In [21]:
for keys in ContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
86    The Olympic flame starts its final leg to Tokyo. Some suggest this day should never have come
Name: title, dtype: object
177    Pulling off Tokyo 2020 will be a logistical nightmare ... and the clock is ticking
Name: title, dtype: object
186    Australian Open turmoil raises questions for Tokyo Games
Name: title, dtype: object
1605    MLB All-Star Game Fast Facts
Name: title, dtype: object
1698    US Olympic hopefuls will be allowed to protest racial and social injustice at trials
Name: title, dtype: object
1627    Texas Rangers sell over 38,000 tickets to home opener, marking one of the first full-capacity sporting events in a year
Name: title, dtype: object
167    Liverpool's shock loss to Brighton leaves it far behind league leaders Manchester City
Name: title, dtype: object
731    August 7 coronavirus news
Name: title, dtype: object
1697    Baseball's Opening Day reflects a polit

## Content Based Similarity Using SVD


In [22]:
# Performing SVD

svd = TruncatedSVD(n_components=50)
lsa = svd.fit_transform(bag_of_words)

In [23]:
def SVDContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(lsa[news_index])
    b = np.array(lsa[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key =
             lambda kv:(kv[1], kv[0]), reverse=True)

  return sorted_most_similar[:first_n]

SVDContentBasedFiltering('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 0.9999999999999998),
 ('6076fb580b3e8bc9b7792991', 0.9002091162356752),
 ('6077056b0b3e8bc9b7793032', 0.8566435138889767),
 ('6076fbfb0b3e8bc9b77929fc', 0.81756636865551),
 ('6076fde30b3e8bc9b7792b3f', 0.7964735004332066),
 ('6076fbe10b3e8bc9b77929ec', 0.7792268083949438),
 ('6076fc3b0b3e8bc9b7792a28', 0.7605005575075502),
 ('607702d20b3e8bc9b7792e76', 0.7490914071351046),
 ('6076fb1b0b3e8bc9b7792966', 0.748448151401771),
 ('607706000b3e8bc9b7793092', 0.7434297827901186)]

In [24]:
for keys in SVDContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
86    The Olympic flame starts its final leg to Tokyo. Some suggest this day should never have come
Name: title, dtype: object
1627    Texas Rangers sell over 38,000 tickets to home opener, marking one of the first full-capacity sporting events in a year
Name: title, dtype: object
193    James Naismith: Basketball inventor is celebrated with Google Doodle
Name: title, dtype: object
505    Eight months pregnant athlete clinches Taekwondo gold medal
Name: title, dtype: object
177    Pulling off Tokyo 2020 will be a logistical nightmare ... and the clock is ticking
Name: title, dtype: object
237    Canada's Trudeau congratulates L.P. Ladouceur of Dallas Cowboys on pending NFL record
Name: title, dtype: object
1285    Bloomberg and The Washington Post are betting big on gaming. Here's why
Name: title, dtype: object
44    RugbyX: 'Rock-and-roll' five-a-side rugby seeking to capitalize on Rugby 

## Content Based Similarity Using LDA


In [25]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
lda = LatentDirichletAllocation(learning_method='batch', n_jobs=-1)

In [26]:
bag_of_words.T

<28376x1761 sparse matrix of type '<class 'numpy.float64'>'
	with 519557 stored elements in Compressed Sparse Column format>

In [27]:
# LDA Cross-Validation
n_components = [20, 50, 70]
learning_decay = [0.5, 0.7, 0.8]


params = {'n_components': n_components, 'learning_decay': learning_decay}

model = GridSearchCV(lda, param_grid=params)
model.fit(bag_of_words.T)

GridSearchCV(estimator=LatentDirichletAllocation(n_jobs=-1),
             param_grid={'learning_decay': [0.5, 0.7, 0.8],
                         'n_components': [20, 50, 70]})

In [28]:
best_params = model.best_estimator_
best_params

LatentDirichletAllocation(n_components=20, n_jobs=-1)

In [29]:
lda_res = best_params.components_.T

In [30]:
lda_res.shape

(1761, 20)

In [31]:
import pickle

pickle_file = 'lda_cross_validation_rev.pkl'

with open(pickle_file, 'wb') as file:
    pickle.dump(model, file)

In [33]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': '19AI35wfuabh1JQ6b1Z3YH5uJ6uL3N6BD'}]})
file.SetContentFile(pickle_file)
file.Upload()

In [34]:
with open(pickle_file, 'rb') as file:
    lda_pkl_model = pickle.load(file)

In [35]:
def LDAContentBasedFiltering(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(lda_res[news_index])
    b = np.array(lda_res[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key =
             lambda kv:(kv[1], kv[0]), reverse=True)

  return sorted_most_similar[:first_n]

LDAContentBasedFiltering('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 0.9999999999999999),
 ('6076fc420b3e8bc9b7792a2b', 0.9999129383719152),
 ('6076fb7f0b3e8bc9b77929ac', 0.9998122848029669),
 ('607705c50b3e8bc9b779306b', 0.9997459957913493),
 ('6076fb590b3e8bc9b7792992', 0.999617232885489),
 ('6076fb490b3e8bc9b7792987', 0.9993780355345249),
 ('607703670b3e8bc9b7792eda', 0.9993070252400056),
 ('6076fbff0b3e8bc9b7792a00', 0.9990971898109428),
 ('607702620b3e8bc9b7792e2f', 0.9988559540000983),
 ('6076fb080b3e8bc9b7792959', 0.9987116036661219)]

In [36]:
for keys in LDAContentBasedFiltering('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
240    Rugby faces 'walking time bomb' as players pursue legal action over degenerative brain disease
Name: title, dtype: object
113    'They are like a family': How esports has helped its fans get through lockdown
Name: title, dtype: object
1667    Major League Baseball is back with a 162-game schedule and actual humans in the stands
Name: title, dtype: object
87    Fierce derby between Japan and South Korea sparking more concern than excitement
Name: title, dtype: object
76    From Manchester United to fruit delivery: David Moyes' long journey to the East End of London
Name: title, dtype: object
1368    Crenshaw says he'll be 'effectively blind' and 'off the grid' for weeks after emergency eye surgery
Name: title, dtype: object
197    Celtic draws with Hibernian after 13 players forced to self-isolate
Name: title, dtype: object
1214    'Black-ish' star Marsai Martin, who's 16, has set a 

## Word embedding using Glove Vectorizor

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/MyDrive/NLP and Text Analysis/glove.6B/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [39]:
i = 0
glov_stop = []
news_embedding_dict = dict()
for word in vectorizor.vocabulary_.keys():
  if word in embeddings_dictionary:
    news_embedding_dict[word] = embeddings_dictionary[word]
  else:
    glov_stop.append(word)


In [44]:
stopset = set(nltk.corpus.stopwords.words('english'))
new_stopwords_list = list(stopset.union(glov_stop))
vectorizor_glov = TfidfVectorizer(stop_words=new_stopwords_list)

In [45]:
glov_bag_of_words = vectorizor_glov.fit_transform(X)

In [46]:
y = np.array([val for (key, val) in news_embedding_dict.items()])

In [47]:
y.shape

(25451, 100)

In [48]:
glov_bag_of_words.shape

(1761, 25451)

In [49]:
document_embedding = glov_bag_of_words*y

In [50]:
document_embedding.shape

(1761, 100)

In [51]:
document_embedding

array([[-0.53041168,  0.84485245,  1.56856967, ..., -1.33973968,
         0.22233791,  0.97958523],
       [-0.10163722,  0.34461491,  0.529451  , ..., -0.76417394,
        -0.45958729,  0.93279579],
       [ 0.3029751 ,  0.52948425,  1.17563666, ..., -1.98697267,
         1.38588292,  0.71529229],
       ...,
       [-0.11292888, -0.41749964,  1.0112224 , ..., -1.30664828,
         0.62894542,  0.76078129],
       [-0.18348154, -0.26512318,  0.94671609, ..., -1.63181234,
        -0.01069463,  0.88530393],
       [-0.80610439,  0.24544622,  0.22215923, ..., -1.56093363,
         0.44395637,  0.89635971]])

In [52]:
def ContentBasedFilteringWordEmbedding(id, first_n = 10):
  similarity_dic = {}
  news_index = data[data['_id']==id].index[0]
  for i in data['_id']:
    an_index = data[data['_id']==i].index[0]
    a = np.array(document_embedding[news_index])
    b = np.array(document_embedding[an_index])
    similarity_dic[i] = similarity(a, b)

  sorted_most_similar = sorted(similarity_dic.items(), key =
             lambda kv:(kv[1], kv[0]), reverse=True)

  return sorted_most_similar[:first_n]

ContentBasedFilteringWordEmbedding('6076fadb0b3e8bc9b779293e')

[('6076fadb0b3e8bc9b779293e', 1.0),
 ('607702e70b3e8bc9b7792e85', 0.8639922524165526),
 ('6076fb580b3e8bc9b7792991', 0.8613115914417774),
 ('6077049c0b3e8bc9b7792faa', 0.8578126796744333),
 ('6076fbe10b3e8bc9b77929ec', 0.8467095558702198),
 ('6077029b0b3e8bc9b7792e53', 0.8432976495149257),
 ('6076fbd20b3e8bc9b77929e2', 0.841979876545199),
 ('6076ff760b3e8bc9b7792c42', 0.8402600830282643),
 ('6077042f0b3e8bc9b7792f61', 0.8397059760670341),
 ('607704e00b3e8bc9b7792fd7', 0.8394334491229545)]

In [71]:
for keys in ContentBasedFilteringWordEmbedding('6076fadb0b3e8bc9b779293e'):
  print(data[data['_id'] == keys[0]]['title'])

4    Brisbane is the 'preferred host' for 2032 Olympics, IOC announces
Name: title, dtype: object
1299    Wisconsin Treasurer jumps into crowded Democratic Senate primary as Ron Johnson freezes GOP field
Name: title, dtype: object
86    The Olympic flame starts its final leg to Tokyo. Some suggest this day should never have come
Name: title, dtype: object
1550    Venice and Florence demand a curb on Airbnb
Name: title, dtype: object
177    Pulling off Tokyo 2020 will be a logistical nightmare ... and the clock is ticking
Name: title, dtype: object
1250    Let's stop telling celebrities to stay out of politics
Name: title, dtype: object
167    Liverpool's shock loss to Brighton leaves it far behind league leaders Manchester City
Name: title, dtype: object
732    August 4 coronavirus news
Name: title, dtype: object
1477    The bitter battle over tourism at Russia's 'Sacred Sea'
Name: title, dtype: object
1570    5 things to know for April 12: Coronavirus, Congress, police violence, Iran,

## Conclusion on which algorithm to go ahead with for content based filtering

We compared Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), and GloVe embeddings for content-based filtering in our news recommender system. Based on our evaluation, LSA consistently provided the most effective and accurate recommendations. Its ability to uncover latent semantic structures in the article dataset proved superior to LDA and GloVe embeddings for our specific application.

# Collaborative Filtering

**Before Starting Collaborative Filtering**

In this section, we explore Collaborative Filtering, a powerful technique in recommendation systems. Unlike Content-Based Filtering, which relies on the characteristics of items and users' past interactions with them, Collaborative Filtering focuses on the collective behavior of users. By analyzing user-item interactions across a dataset, Collaborative Filtering identifies patterns and similarities among users to make personalized recommendations.

**Matrix Factorization Approach**:
For our implementation, we utilized matrix factorization, a common method within Collaborative Filtering. Matrix factorization decomposes the user-item interaction matrix into lower-dimensional matrices, capturing latent factors that represent user preferences and item characteristics. This approach helps us predict how users would rate or interact with items they haven't seen based on similar users' behaviors.

**Data Synthesis**:
To simulate real-world user interactions and evaluate our recommender system, we synthesized our dataset. We created a user-item interaction matrix based on user preferences for randomly selected topics. Articles were served to users with simulated click probabilities and varying time spent, mimicking actual user engagement patterns. This synthesized data allowed us to test and validate our Collaborative Filtering approach effectively.

Next, let's dive into the implementation details of Collaborative Filtering using matrix factorization and analyze its performance in recommending personalized news articles.


In [53]:
topics = ['cricket', 'football', 'golf', 'asia', 'africa', 'europe', 'americas', 'style', 'tech', 'science', 'hollywood', 'us politics', 'stock market', 'travel', 'coronavirus', 'black lives matter']
from random import sample
class User:
  def __init__(self, id):
    self.id = id
    self.prefered_categories = sample(topics, np.random.randint(low=3, high= 5))
    self.no_of_articles_served = np.random.randint(10, 50)*10
    self.no_of_sessions = math.ceil((self.no_of_articles_served)/10)
    self.ids = [self.id for _ in range(self.no_of_articles_served)]
    self.sessions = []
    self.articles_served = []
    self.ratings = []
    self.click = []
    self.ranks = []
    j = math.ceil(self.no_of_articles_served*0.7)
    for m in range(j):
      id_temp = np.random.choice(data[data['topics'].isin(self.prefered_categories)]['_id'])
      self.articles_served.append(id_temp)
      click = np.random.binomial(1, 0.7,1)[0]
      self.click.append(click)
      self.ratings.append('-' if click == 0 else np.random.randint((data[data['_id'] == id_temp]['Max_Time'])/4, data[data['_id'] == self.articles_served[m]]['Max_Time'])[0])

    j = self.no_of_articles_served-j
    for m in range(j):
      id_temp = np.random.choice(data[~data['topics'].isin(self.prefered_categories)]['_id'])
      self.articles_served.append(id_temp)
      click = np.random.binomial(1, 0.1,1)[0]
      self.click.append(click)
      self.ratings.append('-' if click == 0 else np.random.randint(0, data[data['_id'] == id_temp]['Max_Time'])[0])
    for i in range(self.no_of_sessions):
      for k in range(10):
        self.sessions.append(i)
        self.ranks.append(k)


In [54]:
new_user = User(1)
data[data['_id'].isin(new_user.articles_served)].tail(10)

Unnamed: 0                       _id  \
1640        1801  607705880b3e8bc9b7793043   
1650        1813  607705990b3e8bc9b779304f   
1655        1821  607705a70b3e8bc9b7793057   
1659        1826  607705ac0b3e8bc9b779305c   
1660        1827  607705ad0b3e8bc9b779305d   
1672        1847  607705cb0b3e8bc9b7793071   
1673        1848  607705cc0b3e8bc9b7793072   
1676        1851  607705d40b3e8bc9b7793075   
1683        1861  607705e30b3e8bc9b779307f   
1755        1944  607706630b3e8bc9b77930d2   

                                                                                                            title  \
1640                                                Some Americans are acting like the pandemic is over. It's not   
1650                                  Democratic-led states expand voting rights amid GOP push to restrict access   
1655                                        PGA Tour plans to keep its season-ending Tour Championship in Georgia   
1659                              Manufacturing moonshot: How Pfizer makes its millions of Covid-19 vaccine doses   
1660                                What it's like to attend 'Space Club' with astronauts and planetary explorers   
1672  When Covid-19 came for their community, these Choctaw tribal members mobilized to feed and protect families   
1673             Biden administration launches $10 million ad campaign, leaders' network to encourage vaccination   
1676                                                           The latest on the police shooting of Daunte Wright   
1683                                                           Derek Chauvin is on trial for George Floyd's death   
1755                                                                          The voters the GOP wants to silence   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [55]:
def CreateRandomUserProfiler(max_no_user = 40):
  Users = []
  for i in range(max_no_user):
    Users.append(User(i))
    print(Users[i-1].prefered_categories)


  UserProfiler = pd.DataFrame(columns=['UserId', 'SessionID', 'ArticleID Served', 'Article Rank', 'Click', 'Time Spent'])
  for user in Users:
    df = pd.DataFrame()
    df['UserId'] = user.ids
    df['SessionID'] = user.sessions
    df['ArticleID Served'] = user.articles_served
    df['Article Rank'] = user.ranks
    df['Click'] = user.click
    df['Time Spent'] = user.ratings
    UserProfiler = pd.concat([UserProfiler,df], ignore_index=True)

  return UserProfiler

In [56]:
UserProfiler = CreateRandomUserProfiler(40)

['travel', 'us politics', 'science']
['travel', 'us politics', 'science']
['us politics', 'asia', 'black lives matter']
['europe', 'travel', 'style', 'us politics']
['coronavirus', 'europe', 'hollywood']
['science', 'asia', 'black lives matter', 'us politics']
['asia', 'europe', 'tech', 'black lives matter']
['golf', 'tech', 'black lives matter', 'coronavirus']
['hollywood', 'travel', 'golf', 'americas']
['americas', 'style', 'golf']
['hollywood', 'science', 'style', 'americas']
['cricket', 'tech', 'style', 'football']
['golf', 'black lives matter', 'tech']
['travel', 'hollywood', 'tech', 'africa']
['tech', 'hollywood', 'asia']
['travel', 'style', 'europe']
['hollywood', 'golf', 'us politics']
['football', 'americas', 'hollywood']
['europe', 'stock market', 'style']
['americas', 'stock market', 'football']
['coronavirus', 'style', 'africa']
['style', 'science', 'cricket']
['style', 'travel', 'black lives matter']
['football', 'style', 'us politics', 'black lives matter']
['black lives 

In [57]:
UserProfiler.head()

,UserId,SessionID,ArticleID Served,Article Rank,Click,Time Spent
0,0,0,6077035a0b3e8bc9b7792ed2,0,0,-
1,0,0,607704760b3e8bc9b7792f8e,1,1,173
2,0,0,607704970b3e8bc9b7792fa5,2,1,272
3,0,0,6077040e0b3e8bc9b7792f4b,3,1,156
4,0,0,607702e60b3e8bc9b7792e84,4,1,61


In [58]:
UserProfiler.shape

(11600, 6)

## Matrix Factorization

In [59]:
def getNewsInfo(id):
  return data[data['_id']==id]

In [60]:
import numpy as np
from scipy.sparse import csr_matrix

# Creating a user * news sparse matrix
sparseMatrix = csr_matrix((UserProfiler.UserId.unique().shape[0], data.shape[0])).toarray()

In [61]:
k = 0
user = UserProfiler.iloc[k]

for i in UserProfiler.UserId.unique():
  while user.UserId == i and k < UserProfiler.shape[0]:
    user = UserProfiler.iloc[k]
    if user.Click:
      newsInfo = getNewsInfo(user['ArticleID Served'])
      rating = user['Time Spent']/newsInfo['Max_Time']
      sparseMatrix[i][newsInfo.index] = rating
    k+=1

In [62]:
userItem = csr_matrix(sparseMatrix)
from numpy import count_nonzero
sparsity = 1.0 - count_nonzero(sparseMatrix) / sparseMatrix.size
print(sparsity)
pd.DataFrame(sparseMatrix)

0.9311896649630892


,0,1,2,3,4,5,6,7,8,9,...,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.964765,0.507720,0.959628,0.000000,0.000000,0.000000,0.000000,0.868794,0.949153,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.310403,0.000000,0.000000,0.000000,0.756881,0.766808,0.000000,0.429965,0.415254,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.948803,0.994656,0.284404,0.914036,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.611702,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [63]:
def MF(X, num_dims, step_size,epochs,thres,lam_da):
  P = scipy.sparse.rand(X.shape[0], num_dims, 1, format='csr')
  P = scipy.sparse.csr_matrix(P/scipy.sparse.csr_matrix.sum(P, axis=1))
  Q = scipy.sparse.rand(num_dims, X.shape[1], 1, format='csr')
  Q = scipy.sparse.csr_matrix(Q/ scipy.sparse.csr_matrix.sum(Q, axis=0))

  prev_error = 0
  for iterat in range(epochs):
    errors = X - make_sparse(P.dot(Q).todense(), X)
    mse = np.sum(errors.multiply(errors))/len(X.indices)

    P_new=P + step_size*2*(errors.dot(Q.T)-lam_da*P)
    Q_new=Q + step_size*2*(P.T.dot(errors)-lam_da*Q)
    P=P_new
    Q=Q_new
    prev_error=mse
    #if iterat%1==0:
    print(iterat,mse)
  return pd.DataFrame(np.array(P.dot(Q).todense()))



In [64]:
def make_sparse(array, X):
  array = np.array(array)
  x_pos, y_pos = X.nonzero()
  # print(x_pos, y_pos)
  try2 = np.array([[0 for i in range(array.shape[1])] for j in range(array.shape[0])])
  l = len(x_pos)
  for i in range(l):
    # print(x_pos[i], y_pos[i])
    try2[x_pos[i]][y_pos[i]] = array[x_pos[i]][y_pos[i]]

  return scipy.sparse.csr_matrix(try2)

In [65]:
UserItem_MF = MF(userItem, 2, 0.005, 500, 0.00001, 5)

0 0.42244144225366254
1 0.3726350646644539
2 0.29071854021891747
3 0.35037132141218547
4 0.2788724283778372
5 0.36061825960929844
6 0.25435138706638366
7 0.3710503248759104
8 0.2576498340186796
9 0.36244327651301256
10 0.2491707204278803
11 0.37364786708495124
12 0.2666614060248188
13 0.36147300940078975
14 0.25590858204522615
15 0.3815667020869129
16 0.26504445645365204
17 0.3699794699245969
18 0.25946371562063303
19 0.37777338326262383
20 0.26731969842638853
21 0.3718806063917773
22 0.26128412753061886
23 0.3790355673548184
24 0.2615327670927074
25 0.38191809097033347
26 0.2514025109474591
27 0.3936418625228187
28 0.24662500890913908
29 0.40542340864553134
30 0.22896955028337262
31 0.41136426315908636
32 0.24193336141655447
33 0.40545561243328837
34 0.22526942672165837
35 0.4160013335360311
36 0.24460908053428015
37 0.39828675584567125
38 0.22395378783513012
39 0.41675575175446306
40 0.2521343970788195
41 0.40090466760277854
42 0.22240848768108204
43 0.4176205261234406
44 0.251596172

In [68]:
def GetRecommendations(UserID, top_n = 10):
  userRating = UserItem_MF[UserID]
  s = np.array(userRating)
  sort_index = np.argsort(s)
  print(sort_index)
  print('Last n topics',data.iloc[sort_index[-top_n:]]['topics'])
  print('First n topics',data.iloc[sort_index[:top_n]]['topics'])
  recommended_ids = data.iloc[sort_index[-top_n:]]['_id']
  return recommended_ids

In [69]:
ids = GetRecommendations(1)

[30 14 22 29 24  9 25 19  3 16 17  0  8 38 18 15 37 34  4 36 39  6  5  7
 26  1 32 33 23 27 21 31 20 13  2 28 11 12 10 35]
Last n topics 21    cricket
31    cricket
20    cricket
13    cricket
2     cricket
28    cricket
11    cricket
12    cricket
10    cricket
35    cricket
Name: topics, dtype: object
First n topics 30    cricket
14    cricket
22    cricket
29    cricket
24    cricket
9     cricket
25    cricket
19    cricket
3     cricket
16    cricket
Name: topics, dtype: object


In [91]:
for id in ids:
  print(getNewsInfo(id))

    Unnamed: 0                       _id  \
21          21  6076faf90b3e8bc9b779294f   

                                                               title  \
21  Handcuffs to light bulbs shine light on painful Black experience   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# Hybrid Approach using both content based and collaborative based responses

In [89]:
def getHybridRecommendations(user_id, last_read_article, top_n = 10):
  return GetRecommendations(user_id, math.ceil(top_n/2)).tolist() + [item[0] for item in SVDContentBasedFiltering(last_read_article, int(top_n/2))]


In [92]:
hybrid_ids = getHybridRecommendations(1, '6076faec0b3e8bc9b7792947')

[30 14 22 29 24  9 25 19  3 16 17  0  8 38 18 15 37 34  4 36 39  6  5  7
 26  1 32 33 23 27 21 31 20 13  2 28 11 12 10 35]
Last n topics 28    cricket
11    cricket
12    cricket
10    cricket
35    cricket
Name: topics, dtype: object
First n topics 30    cricket
14    cricket
22    cricket
29    cricket
24    cricket
Name: topics, dtype: object


In [94]:
for id in hybrid_ids:
  print(getNewsInfo(id))

    Unnamed: 0                       _id  \
28          28  6076fb000b3e8bc9b7792956   

                                                     title  \
28  Trailblazing Paralympian Margaret Maughan dies aged 91   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  